# Topic Extraction in Newsfeeds
* Autor: Prof. Dr. Johannes Maucher
* Datum: 17.11.2015

[Übersicht Ipython Notebooks im Data Mining Praktikum](Data Mining Praktikum.html)

# Einführung
## Lernziele:

In diesem Versuch sollen Kenntnisse in folgenden Themen vermittelt werden:

* __RSS Feeds:__ Struktur von RSS Feeds analysieren und parsen mit dem _Universal Feed Parser_. 
* __Dokument Analyse:__ Die Häufigkeit aller Worte in einem Dokument (Inhalt des RSS Feeds) zählen und in einem Array verwalten. 
* __Merkmalsextraktion:__ Bestimmung von Merkmalen (hier auch: __Topics__) (Allgemein spricht man von Merkmalen. Im Fall, dass die NNMF auf Dokumente angewandt wird, werden die Merkmale auch mit __Topics__ oder __Themen__ bezeichnet) mit der \emph{Non Negative Matrix Factorization}.
* __Zuordnung__: Wie setzen sich die Topics aus den Wörtern zusammen? Wie stark sind die gefundenen Topics in den Artikeln vertreten?
* __Dokument Clustering:__ Mit der NNMF kann auch ein Clustering realisiert werden. Jeder Topic repräsentiert ein Cluster. Jedes Dokument wird dem Cluster zugeordnet, dessen Topic am stärksten in ihm vertreten ist. 

Sämtliche Verfahren und Algorithmen werden in Python implementiert.

## Theorie zur Vorbereitung

Stellen Sie sich vor Sie möchten in eine eigene Webseite die RSS Feeds einer Menge von Nachrichtenservern einbinden. Da die unterschiedlichen Server wahrscheinlich Artikel zu den gleichen Themen anbieten, werden die Inhalte einiger Artikel ähnlich sein. Mit der __Nicht Negativen Matrixfaktorisierung (NNMF)__ kann für eine große Menge von Dokumenten eine Menge von Themen (Topics) ermittelt werden, auf die sich die Dokumente beziehen. Damit ist es u.a. möglich
* die Dokumente thematisch zu ordnen
* zu jedem Thema nur ein Dokument anzuzeigen

### Ähnlichkeiten bestimmen und relevante Merkmale extrahieren

Eine Sammlung von Dokumenten - in diesem Versuch die Menge aller Nachrichten der angegebenen Feeds - kann in einer Artikel/Wort-Matrix repräsentiert werden. Jede Zeile dieser Matrix gehört zu einem Dokument. Für jedes Wort, das mindestens in einem der Dokumente vorkommt, ist eine Spalte vorgesehen. Das Matrixelement in Zeile $i$, Spalte $j$ beschreibt wie häufig das Wort in Spalte $j$ im zur Zeile $i$ gehörenden Dokument vorkommt.

Unter der Annahme, dass Artikel umso ähnlicher sind, je mehr Worte in diesen gemeinsam vorkommen, kann auf der Grundlage dieser Matrix die Ähnlichkeit zwischen den Artikeln berechnet werden. Hierzu könnte die Matrix z.B. einfach einem _Hierarchischen Clustering_ übergeben werden. Das hierarchische Clustering weist jedoch im Fall einer großen Menge von zu vergleichenden Objekten zwei wesentliche Nachteile auf: Erstens ist die wiederholte Berechnung der Distanzen zwischen allen Artikeln/Clustern extrem rechenaufwendig, zweitens ist die Darstellung einer großen Anzahl von Objekten im Dendrogramm nicht mehr übersichtlich. 

Für das Auffinden von Assoziationen zwischen Dokumenten hat sich in den letzten Jahren die Methode der __Nicht-Negativen Matrix Faktorisierung (NNMF)__ etabliert. Mit dieser Methode kann eine Menge von wesentlichen Merkmalen berechnet werden, anhand derer sich die Dokumente clustern lassen, d.h. Dokumente des gleichen Clusters repräsentieren das gleiche Merkmal (Thema). Ein solches Merkmal wird durch eine Menge von Worten beschrieben, z.B. $\{$ _Paris, terror, IS_ $\}$  oder $\{$_refugee, syria, border_ $\}$. Neben der Merkmalsextraktion stellt die relativ geringe Komplexität einen weiteren Vorteil der NNMF dar. Durch die Darstellung der Artikel/Wort-Matrix als Produkt von 2 Faktormatrizen müssen deutlich weniger Einträge gespeichert werden.

### Nicht Negative Matrixfaktorisierung: Die Idee

Die Artikel/Wort-Matrix wird im Folgenden mit $A$ bezeichnet. Sie besitzt $r$ Zeilen und $c$ Spalten, wobei $r$ die Anzahl der Artikel und $c$ die Anzahl der relevanten Worte in der Menge aller Artikel ist. Durch Multiplikation der Matrix $A$ mit dem Vektor $v$ (_wordvec_: Vektor der alle relevanten Worte enthält) werden die Worte den Artikeln $a$ (_articletitles_: Vektor der alle Artikeltitel enthält) zugeordnet:

$$
a=A*v.
$$

Die Idee der NNMF besteht darin die Matrix $A$ als Produkt zweier Matrizen $W$ und $H$ darzustellen,

$$
A=W*H
$$

wobei alle Elemente in $W$ und $H$ größer oder gleich Null sein müssen. Die Matrixmultiplikation erfordert, dass die Anzahl der Zeilen $m$ in $H$ gleich der Anzahl der Spalten in $W$ sein muss. 
Durch die Faktorisierung der Matrix $A$ wird die Zuordnung der Wörter des Wortvektors $v$  zu den Artikeln des Vektors $a$ in zwei Stufen zerlegt. 

$$
f = H*v
$$
$$
a = W*f 
$$

In der ersten Stufe werden durch die Multiplikation von $v$ mit der Matrix $H$ die Wörter einem sogenannten Merkmalsvektor $f$ mit $m$ Elementen zugewiesen. In der zweiten Stufe werden durch die Multiplikation des Merkmalsvektor $f$ mit der Matrix $W$ die einzelnen Merkmale den Artikeln in $a$ zugeordnet. Die Matrix $H$ definiert also aus welchen Wörtern die Merkmale gebildet werden. Sie wird deshalb __Merkmalsmatrix__ genannt. Die Matrix $W$ hingegen beschreibt mit welchem Gewicht die einzelnen Merkmale in den verschiedenen Artikeln auftreten. Sie wird deshalb __Gewichtungsmatrix__ genannt.

Daraus folgt: Wenn eine Faktorisierung der Matrix $A$ gefunden wird, dann werden damit auch relevante Merkmale, also die Themen, definiert, hinsichtlich derer die Artikel effizient kategorisiert werden. Durch die Matrixfaktorisierung wird eine __Merkmalsextraktion__ realisiert. 

### Berechnung der Matrixfaktoren

Für die Berechnung der Faktoren wurde in [Lee, Algortihms for Non-negative Matrix Factorisation](http://papers.nips.cc/paper/1861-algorithms-for-non-negative-matrix-factorization.pdf) eine iterative Methode vorgestellt, die derzeit wohl am häufigsten angewandt wird und auch in dieser Übung implementiert werden soll. Der Algorithmus besteht aus folgenden Schritten:
* Gebe die zu faktorisierende Matrix $A$ ein. $r$ sei die Anzahl der Zeilen und $c$ die Anzahl der Spalten von $A$.
* Wähle die Anzahl $m$ der Merkmale, mit $m<c$. _Tipp:_ Für $m$ sollte zunächst ein Wert im Bereich $15$ bis $30$ gewählt werden.
* Lege eine $m \times c$ Matrix $H$ an mit initial zufälligen Elementen (Anwendung der numpy Funktion _random.random()_)
* Lege eine $r \times m$ Matrix $W$ an mit initial zufälligen Elementen (Anwendung der numpy Funktion _random.random()_)
* Wiederhole bis maximale Anzahl der Iteration erreicht oder Kosten $k$ unter vordefinierter Schwelle:

	* Berechne aktuelles Produkt $B=W*H$ und bereche die Kostenfunktion 
		$$
			k=\left\| A - B \right\|^2 = \sum\limits_{i,j} \left(A_{i,j} - B_{i,j}\right)^2
		$$ 
	* Anpassung der Matrix $H$ durch folgende Neuberechnung der Matrixelemente
    
		$$
		H_{i,j} := H_{i,j} \frac{(W^T*A)_{i,j}}{(W^T*W*H)_{i,j}}
		$$
        
	* __Nach__ der Anpassung der Matrix $H$: Anpassung der Matrix $W$ durch folgende Neuberechnung der Matrixelemente
    
		$$
		W_{l,i} := W_{l,i} \frac{(A*H^T)_{l,i}}{(W*H*H^T)_{l,i}}
		$$

In [Lee, Algortihms for Non-negative Matrix Factorisation](http://papers.nips.cc/paper/1861-algorithms-for-non-negative-matrix-factorization.pdf) ist bewiesen, dass durch die o.g. Anpassungsroutinen die Kosten $k$ monoton abnehmen und in einem Minimum konvergieren. Der Algorithmus ist jedoch nicht optimal weil das gefundene Minimum ein lokales Minimum sein kann.

 ## Vor dem Versuch zu klärende Fragen
 
 * Was versteht man unter Artikel/Wort-Matrix? Wie wird diese im aktuellen Versuch gebildet?

Eine Artikel/Wort Matrix besteht aus r Zeilen, die für die Artikel stehen und c Spalten die für alle Wörter in den Artikeln stehen. Die Werte innerhalb einer Zeile stehen für die Häufigkeit der jeweiligen Worte.

* Wie multipliziert man die Matrix
    $$
    A= \left( \begin{array}{cccc}
a_{00} & a_{01} & a_{02} & a_{03} \\ 
a_{10} & a_{11} & a_{12} & a_{13} \\ 
a_{20} & a_{21} & a_{22} & a_{23}
\end{array} \right)
    $$
    mit dem Vektor  
    $$
    v=\left( \begin{array}{c}
v_{0} \\ 
v_{1} \\ 
v_{2} \\ 
v_{3}
\end{array} \right)
    $$
    

a = A &ast; v 


a = (a00&ast;v0 + a01&ast;v1 + a02&ast;v2 + a03&ast;v3)
    (a10&ast;v0 + a11&ast;v1 + a12&ast;v2 + a13&ast;v3) 
    (a20&ast;v0 + a21&ast;v1 + a22&ast;v2 + a23&ast;v3)

* Was versteht man im Kontext der NNMF unter
    * Merkmalsmatrix
    * Gewichtsmatrix

Die Merkmalsmatrix ist eine Merkmal x Wort Matrix, die für ein Merkmal die Wörter festlegt, die dieses Mermal ausmachen.
Die Gewichtsmatrix ist eine Artikel x Merkmals Matrix, die für jeden Artikel die Gewichtung der einzelnen Mermale bestimmt, d.h. wie stark ein Merkmal in dem Artikel vertreten ist

* Wie definieren die Zeilen der Merkmalsmatrix die einzelnen Merkmale (Topics)?
* Was definieren die Zeilen der Gewichtungsmatrix?

Eine Zeile der Mermalsmatrix steht für ein gefundenes Merkmal.
Eine Zeile steht der Gewichtungsmatrix steht für einen bestimmten Artikel.

* Wie werden in Numpy zwei Arrays (Typ numpy.array) 
	* im Sinne der Matrixmultiplikation miteinander multipliziert?
	* elementweise multipliziert?
* Wie wird die Transponierte eines Numpy-Arrays berechnet?

In [3]:
import numpy as np

a = np.matrix([[1, 1], 
               [1, 1]])

b = np.matrix([[0, 1], 
               [2, 0]])

# Matrixmultiplikation
print a * b

# Elementweise Multiplikation
print np.multiply(a, b)

# Matrix transponieren
print b.transpose()

[[2 1]
 [2 1]]
[[0 1]
 [2 0]]
[[0 2]
 [1 0]]


# Versuchsdurchführung
Die in diesem Versuch einzubindenden Feeds sind in der unten stehenden Liste _feedlist_ definiert. Die aus dem vorigen Vesuch bereits bekannte Funktion _stripHTML()_ ist ebenfalls gegeben:

In [4]:
import feedparser

feedlist=['http://feeds.reuters.com/reuters/topNews',
          'http://feeds.reuters.com/reuters/businessNews',
          'http://feeds.reuters.com/reuters/worldNews',
          'http://feeds2.feedburner.com/time/world',
          'http://feeds2.feedburner.com/time/business',
          'http://feeds2.feedburner.com/time/politics',
          'http://rss.cnn.com/rss/edition.rss',
          'http://rss.cnn.com/rss/edition_world.rss',
          'http://www.nytimes.com/services/xml/rss/nyt/GlobalHome.xml',
          'http://feeds.nytimes.com/nyt/rss/Business',
          'http://www.nytimes.com/services/xml/rss/nyt/World.xml',
          'http://www.nytimes.com/services/xml/rss/nyt/Economy.xml'
          ]

In [5]:
def stripHTML(h):
  p=''
  s=0
  for c in h:
    if c=='<': s=1
    elif c=='>':
      s=0
      p+=' '
    elif s==0: p+=c
  return p

## Anlegen der Artikel/Wort-Matrix

### Die Funktion _getarticlewords()_
Schreiben Sie eine Funktion _getarticlewords()_, die folgende Elemente zurückgibt:

* _allwords:_ ist ein Dictionary dessen Keys die Worte aller gesammelten Artikel sind. Der zu jedem Key gehörende Wert ist die Anzahl, wie oft das Wort insgesamt vorkommt.
* _articlewords:_ ist eine Liste mit so vielen Elementen wie Artikel in der Sammlung sind. Jedes Listenelement ist ein Dictionary, welches die Worte des jeweiligen Artikels als Key enthält und als Wert die Worthäufigkeit.
* _articletitles_ ist eine Liste mit so vielen Elementen wie Artikel in der Sammlung sind. Jedes Element ist der Artikeltitel als String.

Für das Parsing der Feeds soll wieder das Modul _feedparser_ eingesetzt werden. Die zu einer Nachricht gehörenden Wörter sollen die Wörter des Elements _title_ und die Wörter des Elements _description_ sein (siehe voriger Versuch). Allerdings sollen hier nicht alle Wörter eingebunden werden, sondern wie im vorigen Versuch eine Methode _getwords()_ implementiert werden, welche nur die _relevanten_ Wörter zurückgibt. Die Frage welche Wörter relevant sind ist nicht eindeutig beantwortbar. Sie können sich hierzu eigene Antworten einfallen lassen. Auf jeden Fall sollten aber die Stopwörter ignoriert werden. Hierzu kann z.B. die Stopwortliste von NLTK angewandt werden.

Nachdem alle relevanten Wörter aller Nachrichten gesammelt sind, sollte eine weitere Bereinigung stattfinden, die 

* alle Wörter, die weniger als 4 mal vorkommen
* alle Wörter, die in mehr als 30% aller Dokumente vorkommen

entfernt. 

Durch dieses Herausfiltern nicht relevanter Wörter kann es vorkommen, dass einzelne Artikel keine relevanten Wörter mehr enthalten. Diese Artikel sollen dann ganz ignoriert werden. D.h. unter anderem, dass diese Artikel auch nicht in _articlewords_ und _articletitles_ erscheinen.

In [6]:
from nltk.corpus import stopwords
from nltk import word_tokenize
from itertools import dropwhile
from nltk.stem import WordNetLemmatizer

wnl = WordNetLemmatizer()

custom_stopwords = ['said', 'could', 'according', 'across', 'first', 'reuters']


weekdays = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
time = ['today', 'yesterday', 'tomorrow', 'day', 'month', 'year', 'decade']
miscellaneous = ['said', 'could', 'according', 'across', 'first', 'reuters']

# custom_stopwords.extend(weekdays)
# custom_stopwords.extend(time)
# custom_stopwords.extend(miscellaneous)

# Remove stopwords and transform to lowercase
def getwords(text):
    
    stop = set(stopwords.words('english'))
    
#     for i in custom_stopwords:
#         stop.add(i)
    
    stop.update(custom_stopwords)
    return [wnl.lemmatize(i.lower()) for i in word_tokenize(text) if i.lower() not in stop and len(i) > 4]

def findIrrelevant(words, min_frequency, max_percentage):
    word_sum = sum(words.values())
    return {word for word, frequency in words.items() if frequency <= min_frequency or float(frequency)/word_sum > max_percentage}

def getarticlewords(feed):
    allwords = dict()
    articlewords = list()
    articletitles = list()
    f=feedparser.parse(feed)
    for article in f.entries:
        if 'description' in article and 'title' in article:
            fulltext = stripHTML(article.title + ' ' + article.description)
            words = getwords(fulltext)
            articleword = dict()
            for word in words:
                if word in articleword:
                    articleword[word] += 1
                else:
                    articleword[word] = 1
                    
                if word in allwords:
                    allwords[word] += 1
                else:
                    allwords[word] = 1
            articlewords.append(articleword)
            articletitles.append(article.title)
            
    irrelevant_words = findIrrelevant(allwords, 2, 0.5)
    
    for word in irrelevant_words:
        del allwords[word]
        
        for articleword in articlewords:
            if word in articleword:
                del articleword[word]
                
    if not allwords:
        return None
    
    articlewords, articletitles = zip(*[(articleword, title) for articleword, title in zip(articlewords, articletitles) if articleword])
        
    return (allwords, np.array(articlewords), np.array(articletitles))

In [7]:
print getwords("ASDASDASDASD")

['asdasdasdasd']


### Die Funktion _makematrix()_
Schreiben Sie eine Funktion _makematrix()_, die aus dem Dictionary _allwords_ und der Liste _articlewords_ (vorige Aufgabe) die Artikel-/Wort-Matrix generiert. Die Einträge in der Matrix sollen die Häufigkeiten der Wörter im jeweiligen Dokument sein (term frequency tf). Die Artikel-/Wort-Matrix soll als 2-dimensionales Numpy Array angelegt werden.

In [8]:
def makematrix(allwords, articlewords):
    words = allwords.keys()
    article_word_matrix = np.zeros((len(articlewords), len(words)))
    for i in range(len(article_word_matrix)):
        row = article_word_matrix[i]
        current_articlewords = articlewords[i]
        for word in current_articlewords.keys():
            row[words.index(word)] = current_articlewords[word]
    return (np.matrix(article_word_matrix), words)

## Die Nicht Negative Matrix Faktorisierung
Die Implementierung der NNMF ist entsprechend der Beschreibung im Theoriekapitel durchzuführen.

* Implementieren Sie die Funktion _cost(A,B)_. Dieser Funktion werden zwei Numpy-Matrizen $A$ und $B$ übergeben. Zurück geliefert werden die nach oben angegebener Formel berechneten Kosten $k$. Diese Funktion wird von der im folgenden beschriebenen Funktion _nnmf(A,m,it)_ benutzt.
* Implementieren Sie die Funktion __nnmf(A,m,it)__. In dieser Funktion soll der oben beschriebene Algorithmus für die Nicht-negative Matrix Faktorisierung ausgeführt werden. Der Funktion wird die zu faktorisierende Matrix $A$, die Anzahl der Merkmale $m$ und die Anzahl der Iterationen $it$ übergeben. Die Funktion gibt die gefundenen Faktoren $W$ und $H$ zurück. In jeder Iteration sollen mit der Funktion __cost(A,B)__ die Kosten berechnet werden. Sobald die Kostenabnahme pro 10 Iterationen kleiner als $2$ ist oder eine maximale Anzahl von Iterationen ($maxIt=200$) erreicht ist, soll der Algorithmus mit der Rückgabe der Faktoren $W$ und $H$ terminieren.     


Tipp für die Implementierung elementweiser Operationen von Matrizen: Für elementweise Operationen müssen in Python/Numpy nicht alle Elemente über Schleifen explizit berechnet werden. Eine elementweise Anpassung aller Matrixelemente kann kompakt programmiert werden indem die beteiligten Matrizen für diese Operationen als Arrays implementiert werden. Sollen z.B. die beiden gleich großen Numpy Arrays $U$ und $V$ elementweise multipliziert werden, dann wäre der entsprechende Programmcode einfach _U*V_.  

In [9]:
def cost(A, B):
    return np.sum(np.square(A - B))
    
def nnmf(A, m, it, threshold=2):
    (r, c) = A.shape
    
    if not m < c:
        raise ValueError()
         
    B = None
    
    cost_iteration = 0
    cost_start = None
    cost_current = None
    
    H = np.matrix(np.random.random((m, c)))
    W = np.matrix(np.random.random((r, m)))

    for i in range(it): 
        cost_iteration += 1
        
        if cost_iteration == 10:
            if cost_start - cost_current > threshold:
                cost_iteration = 0
                cost_start = cost_current
            else:
                break
      
        B = W * H
        
        _cost = cost(A, B)
        
        if not cost_start:
            cost_start = _cost
        
        cost_current = _cost
        
        H = np.multiply(H, (W.T * A)/(W.T*W*H))
        W = np.multiply(W, (A * H.T)/(W*H*H.T))
        
        if it - 1 == i:
            print "Reached maximum amount of iterations"
    
    return (W, H)

from sklearn.decomposition import NMF

def nnmfsk(A, m, it):
    nmf_model = NMF(n_components=m, init='random', random_state=0, max_iter=it)
    W = nmf_model.fit_transform(A)
    H = nmf_model.components_
    return (W, H)

## Anzeige der Merkmale und der Gewichte

Im vorigen Abschnitt wurde die Merkmalsmatrix $H$ und die Gewichtsmatrix $W$ berechnet. Diese Matrizen können natürlich am Bildschirm ausgegeben werden, was jedoch nicht besonders informativ ist. Aus den Matrizen können jedoch die Antworten für die folgenden interessanten Fragen berechnet werden:

* In welchen Artikeln sind welche Merkmale stark vertreten?
* Wie lassen sich die insgesamt $m$ Merkmale beschreiben, so dass aus dieser Merkmalsbeschreibung klar wird, welches Thema den Artikeln, in denen das Merkmal stark vertreten ist, behandelt wird? 
 
Die Antwort auf die erste Frage ergibt sich aus der Gewichtsmatrix $W$. Für die Beantwortung der zweiten Frage wird die Merkmalsmatrix $H$ herangezogen.



### Beschreibung der Merkmale

Die Merkmalsmatrix $H$ beschreibt, wie stark die Worte aus _wordvec_ in jedem Merkmal enthalten sind. Jede Zeile von $H$ gehört zu einem Merkmal, jede Spalte von $H$ gehört zu einem Wort in _wordvec_.

Es bietet sich an jedes Merkmal einfach durch die $N=6$ Wörter aus _wordvec_ zu beschreiben, welche am stärksten in diesem Merkmal enthalten sind. Hierzu muss für jedes Merkmal die entsprechende Zeile in $H$ nach den $N=6$ größten Werten durchsucht bzw. geordnet werden. Die entsprechenden Spalten dieser Matrixelemente verweisen dann auf die $N=6$ wichtigsten Worte des Merkmals.

Tipp für die Implementierung: Legen Sie für jedes Merkmal $i$ eine Liste an. Die Listenlänge ist durch die Anzahl der Worte in _wordvec_ (d.h. die Anzahl der Spalten in $H$) gegeben. Jedes Listenelement $j$ enthält selbst wieder 2 Elemente: An erster Stelle den entsprechenden Wert $H_{i,j}$ der Merkmalsmatrix, an der zweiten Stelle das $j.$-te Wort in _wordvec_. Nachdem die Liste angelegt ist, kann sie mit _listname.sort()_ in aufsteigender Reihenfolge sortiert werden. Die abnehmende Sortierung erhält man mit _listname.sort().reverse()_. Danach geben die $N=6$ ersten Listenelemente die für das Merkmal $i$ wichtigsten Worte an.

   
### Präsenz der Merkmale in den Artikeln

Die Gewichtsmatrix $W$ beschreibt, wie stark die $m$ Merkmale in den Artikeln aus _articletitles_ enthalten sind. Jede Zeile von $W$ gehört zu einem Artikel, jede Spalte von $W$ gehört zu einem Merkmal.
Die Berechnung der $M=2$ gewichtigsten Merkmale für jeden Artikel in _articletitles_ kann analog zu der oben beschriebenen Berechnung der $N=6$ wichtigsten Worte eines Merkmals berechnet werden.


### Implementierung

Implementieren Sie eine Funktion _showfeatures(w,h,titles,wordvec)_, welche wie oben beschrieben für jeden Artikel die $M=2$ wichtigsten Merkmale am Bildschirm ausgibt. Dabei soll jedes Merkmal durch die 6 wichtigsten Wörter dieses Merkmals angegeben werden. Siehe Beispielausgabe unten.  

Übergabeparameter der Funktion sind die Merkmalsmatrix $H$, die Gewichtungsmatrix $W$, die Liste aller Artikeltitel _articletitles_ und die Liste aller Worte _wordvec_.


Beispiel fuer Ausgabe:

[(13.54131155883748, 13, u'Putin vows payback after confirmation of Egypt plane bomb'),

(2.2466669548146254, 9, u'Putin vows payback after confirmation of Egypt plane bomb')]

----- ['plane', 'egypt', 'russia', 'month', 'killing', 'putin']

----- ['airport', 'russian', 'crash', 'egypt', 'security', 'officials']

Die Ausgabe ist wie folgt zu interpretieren:
* Für den Artikel _Putin vows payback after confirmation of Egypt plane bomb_ ist 
    * das wichtigste Merkmal durch die 6 Wörter _plane_, _egypt_, _russia_, _month_, _killing_, _putin_ definiert. Das Gewicht dieses Merkmals im Artikel ist 13.54
    * das zweitwichtigste Merkmal durch die 6 Wörter _airport_, _russian_, _crash_, _egypt_, _security_, _officials_ definiert. Das Gewicht dieses Merkmals im Artikel ist 2.24

In [10]:
def getarticlefeatures(w, titles):
    return zip([sorted(zip(article, range(len(article))), reverse=True)[:2] for article in np.array(w)], titles)


def showfeatures(w, h, titles, wordvec):
    features = [(sorted(zip(feature, wordvec), reverse=True))[:6] for feature in np.array(h)]
    article_features = getarticlefeatures(w, titles)
    for feature_weights, article in article_features:
        print feature_weights, article
        for weight, feature_index in feature_weights:
            print '-------', zip(*features[feature_index])[1]
    return features
    
def runnmf(topic_count, iterations):
    for feed in feedlist:
        article_words = getarticlewords(feed)
        if article_words:
            matrix, wordvec = makematrix(article_words[0], article_words[1])
            if (matrix.shape[1] > 10):
                W, H = nnmf(matrix, topic_count, iterations)
                features = showfeatures(W, H, article_words[2], wordvec)
                

In [17]:
runnmf(6, 50)

[(1.8615466406361942, 5), (1.1139477457289202, 1)] Putin: 'We need friends'
------- (u'election', u'obama', u'campaign', u'trump', u'donald', u'mattis')
------- (u'america', u'obama', u'group', u'mattis', u'campaign', u'donald')
[(2.1377212034011417, 2), (0.0, 5)] Assad: Capturing Aleppo 'huge step'
------- (u'syria', u'aleppo', u'civil', u'year', u'mattis', u'obama')
------- (u'election', u'obama', u'campaign', u'trump', u'donald', u'mattis')
[(4.2651484311899495, 4), (0.0031534305876396735, 0)] Trump's EPA pick is global warming skeptic
------- (u'trump', u'donald', u'campaign', u'obama', u'policy', u'election')
------- (u'foreign', u'policy', u'come', u'america', u'trump', u'donald')
[(2.2408356278732078, 5), (6.543638452457685e-29, 2)] Michigan recount halted
------- (u'election', u'obama', u'campaign', u'trump', u'donald', u'mattis')
------- (u'syria', u'aleppo', u'civil', u'year', u'mattis', u'obama')
[(0.99105933237954558, 0), (2.341098633241997e-25, 2)] UK's Johnson rebuked ove

## Aufgaben

1. Analysieren Sie die berechneten Topics indem Sie sich überlegen ob die gefundenen 6 Wörter pro Topic wirklich Themen beschreiben.
2. Verändern Sie die Parameter der NNMF (Anzahl der Topics $m$, Anzahl der Iterationen). Bei welcher Einstellung der Parameter erhalten Sie das für sie sinnvollste Resultat (sinnvolle Topics)?
3. Wie kann die _getwords()_ Methode verbessert werden, so dass noch bedeutsamere Topics gefunden werden? 

1 -  die gefundenen Wörter beschreiben teilweise tatsächlich Themen, auf der anderen Seite sind die gefundenen Wörter nur aktuell oft besprochen und für das Topic selbst gar nicht relevant. Zum Beispiel haben wir oft "Syrien" und "Trump" in den Ergebnissen, auch wenn diese gar keinen direkten Bezug zum Artikel haben.

### Aufgabe 1
Für eine grobe Einordnung des Artikels reichen 6 Wörter aus. Vor allem bei Artikeln, bei denen ein sehr hohes Gewicht für das jeweilige Topic gefunden wurde (3 oder mehr), beschreiben die Wörter das Thema ziemlich akurat.

Topics, die für einen Artikel weniger gut passen, sind dagegen oft nicht aussagekräftig genug um wirklich genau zu wissen worum es geht. 

### Aufgabe 2

Eine hohe Anzahl von Topics, erhöht im Durchschnitt auch die Gewichte für die einzelnen Artikel. D.h. die gefundenen Topics stimmen besser mit den individuellen Artikeln überein. Problematisch wird hier allerdings eine zu hohe Anzahl, da sonst für jeden Artikel ein einzelnes Topic gefunden wird. 

Zu dem Zeitpunkt des Schreibens, wurde bei maximal 38 Iterationen für jede Matrix eine Faktorisierung gefunden. In diesem Fall ist ein höhrer Wert unnötig, da dieser keine Auswirkung auf das Ergebnis mehr hat. Allerdings kann es natürlich sein, dass bei anderen Dokumenten mehr Iterationen nötig sind. Deshalb haben wir einen Wert von 50 gewählt.

### Aufgabe 3

Bei der Ausgabe der Topics war zunächst auffällig, dass trotz der Entfernung der Stopwörter, bestimmte Wörter wie z.B. 
'reuters' (Name der Nachrichtenagentur), 'today' oder 'said' immernoch in einem Großteil der Topics vorkommen. Diese haben wir manuell entfernt. Dazu kam, dass Topics oft ein Wort und zusätzlich den Plural des Wortes beinhaltet haben. Das Problem lässt sich mit einer Lematisierung lösen. 